In [0]:
%sql
-- TRUNCATE TABLE portfolio_dev.default.test
SELECT * FROM portfolio_dev.default.test

In [0]:
from pyspark.sql.functions import current_timestamp

# Read 1 row from samples.nyctaxi.trips
df = spark.table("samples.nyctaxi.trips").limit(1)

# Add a column with current timestamp
df_with_ts = df.withColumn("ingest_ts", current_timestamp())

# Write to portfolio_dev.default.testit using structured streaming (append mode)
(df_with_ts
    .write
    .option("mergeSchema", "true").saveAsTable("portfolio_dev.default.test", mode="append")
)

In [0]:
# Read from portfolio_dev.default.test as a streaming source
df_stream = (spark.readStream
    .format("delta")
    .table("portfolio_dev.default.test")
)

# Write the streaming data into portfolio_dev.default.test2
(df_stream.writeStream
    .trigger(availableNow=True)
    .format("delta")
    .option("checkpointLocation", "/tmp/checkpoints/portfolio_dev_default_test2")
    .outputMode("append")
    .toTable("portfolio_dev.default.test2")
)